In [122]:
import os
import pandas as pd
import numpy as np
from unidecode import unidecode

import warnings
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

# IMPORTS E FUNCOES

In [123]:
def remover_acentos(texto):
    return unidecode(texto)

In [124]:
def convert_to_int(value):
    try:
        return int(value)
    except (ValueError, TypeError):
        return pd.NA

In [125]:
historico = pd.read_excel(f'../datasets/xlsx/historico(naotratado).xlsx')
tabela = pd.read_excel(f'../datasets/xlsx/tabela(naotratado).xlsx')
rodadas = pd.read_excel(f'../datasets/xlsx/rodadas(naotratado).xlsx')
elencos = pd.read_excel(f'../datasets/xlsx/elenco(naotratado).xlsx')

In [126]:
season_atual = historico['season'].max()
rodada_recente = historico[historico['season'] == season_atual]['round'].str.split(' ', expand= True)[1].astype(int).max()

# HISTORICO

### Data cleaning

In [127]:
historico.drop(
  ['notes','captain', 'formation', 'referee', 'match report', 'attendance', 'time', 'day',
   'cmp.1', 'cmp.2', 'cmp.3', 'att.1', 'att.2', 'att.3', 'tkl.1'], 
  axis= 'columns', inplace= True)

In [128]:
historico[['gf', 'ga']] = historico[['gf', 'ga']].astype(int)
historico['season'] = historico['season'].astype(int)
historico['round'] = historico['round'].str.split(' ', expand= True)[1].astype(int)

In [129]:
historico['date'] = pd.to_datetime(historico['date'], format='%Y-%m-%d')
historico['days'] = (max(historico['date']) - historico['date']).dt.days
historico['time_diff'] = np.exp(-0.001*historico['days'])

In [130]:
historico['opponent'] = historico['opponent'].apply(remover_acentos)
historico['opponent'] = [linha.lower() for linha in historico['opponent']]
historico['opponent'] = historico['opponent'].str.replace(' ', '_')
historico['opponent'] = historico['opponent'].str.replace('(', '')
historico['opponent'] = historico['opponent'].str.replace(')', '')
historico['opponent'] = historico['opponent'].str.replace(')', '')

# Especifico para o brasileirão
historico['opponent'] = historico['opponent'].str.replace('atletico', 'atl')
historico['team'] = historico['team'].str.replace('athletico', 'ath')
historico['team'] = historico['team'].str.replace('atletico', 'atl')

# Especifico para a Premier League
historico['team'] = historico['team'].str.replace('_hotspur', '')
historico['team'] = historico['team'].str.replace('rhampton_wanderers', 's')
historico['team'] = historico['team'].str.replace('united', 'utd')
historico['team'] = historico['team'].str.replace('west_ham_utd', 'west_ham')
historico['team'] = historico['team'].str.replace('_and_hove_albion', '')
historico['team'] = historico['team'].str.replace('wich_albion', '')
historico['team'] = historico['team'].str.replace('_town', '')
historico['team'] = historico['team'].str.replace('luton', 'luton_town')
historico['opponent'] = historico['opponent'].str.replace('united', 'utd')
historico['opponent'] = historico['opponent'].str.replace("nott'ham_forest", "nottingham_forest")

# Especifico para a Serie A TIM
historico['team'] = historico['team'].str.replace('nazionale', '')

# Especifico para a Bundesliga
historico['opponent'] = historico['opponent'].str.replace("m'gladbach", 'monchengladbach')
historico['team'] = historico['team'].str.replace("bayer_leverkusen", 'leverkusen')
historico['team'] = historico['team'].str.replace("eintracht_frankfurt", 'eint_frankfurt')

# Especifico para a La Liga
historico['team'] = historico['team'].str.replace("real_betis", 'betis')



In [131]:
[item for item in historico['opponent'].unique() if item not in historico['team'].unique()]

[]

In [132]:
historico['team'].unique()

array(['botafogo_rj', 'palmeiras', 'bragantino', 'gremio', 'flamengo',
       'atl_mineiro', 'ath_paranaense', 'fluminense', 'sao_paulo',
       'fortaleza', 'internacional', 'cuiaba', 'corinthians', 'santos',
       'bahia', 'vasco_da_gama', 'cruzeiro', 'goias', 'coritiba',
       'america_mg', 'manchester_city', 'tottenham', 'liverpool',
       'arsenal', 'aston_villa', 'newcastle_utd', 'brighton',
       'manchester_utd', 'brentford', 'chelsea', 'crystal_palace',
       'west_ham', 'nottingham_forest', 'wolves', 'fulham', 'everton',
       'luton_town', 'bournemouth', 'burnley', 'sheffield_utd', 'inter',
       'juventus', 'milan', 'napoli', 'atalanta', 'bologna', 'roma',
       'fiorentina', 'monza', 'lazio', 'frosinone', 'torino', 'lecce',
       'genoa', 'sassuolo', 'udinese', 'cagliari', 'hellas_verona',
       'empoli', 'salernitana', 'leverkusen', 'bayern_munich',
       'stuttgart', 'dortmund', 'rb_leipzig', 'hoffenheim',
       'eint_frankfurt', 'freiburg', 'wolfsburg', 'aug

#### Ids dos times

In [133]:
#all_teams = pd.unique(historico[['team', 'opponent']].values.ravel('K'))
#all_leagues = pd.unique(historico['comp'].values.ravel('K'))
#team_mapping, _ = pd.factorize(all_teams)
#league_mapping, _ = pd.factorize(all_leagues)
#
#historico['home_team_id'] = historico['team'].map(dict(zip(all_teams, team_mapping)))
#historico['away_team_id'] = historico['opponent'].map(dict(zip(all_teams, team_mapping)))
#historico['league_id'] = historico['comp'].map(dict(zip(all_leagues, league_mapping)))

In [134]:
#teams_id = historico[['home_team_id', 'team', 'comp']].drop_duplicates().reset_index(drop= True)
#teams_id.rename(columns={'home_team_id': 'team_id',
#                         'team': 'team_name'}, inplace=True)
#teams_id['league_id'] = pd.factorize(teams_id['comp'])[0]
#teams_id = teams_id.drop(['comp'], axis= 1)
#teams_id

In [135]:
historico.reset_index(inplace=True)
historico.rename(columns={'index': 'game_id'}, inplace=True)

In [136]:
teams_id = {
    'team_id': list(range(0, 98)),
    'team_name': [
        'botafogo_rj', 'palmeiras', 'bragantino', 'gremio', 'flamengo', 'atl_mineiro',
        'ath_paranaense', 'fluminense', 'sao_paulo', 'fortaleza', 'internacional',
        'cuiaba', 'corinthians', 'santos', 'bahia', 'vasco_da_gama', 'cruzeiro',
        'goias', 'coritiba', 'america_mg', 'manchester_city', 'tottenham', 'liverpool',
        'arsenal', 'aston_villa', 'newcastle_utd', 'brighton', 'manchester_utd', 'brentford',
        'chelsea', 'crystal_palace', 'west_ham', 'nottingham_forest', 'wolves', 'fulham',
        'everton', 'luton_town', 'bournemouth', 'burnley', 'sheffield_utd', 'inter', 'juventus',
        'milan', 'napoli', 'atalanta', 'bologna', 'roma', 'fiorentina', 'monza', 'lazio', 'frosinone',
        'torino', 'lecce', 'genoa', 'sassuolo', 'udinese', 'cagliari', 'hellas_verona', 'empoli',
        'salernitana', 'leverkusen', 'bayern_munich', 'stuttgart', 'dortmund', 'rb_leipzig',
        'hoffenheim', 'eint_frankfurt', 'freiburg', 'wolfsburg', 'augsburg', 'monchengladbach',
        'werder_bremen', 'heidenheim', 'bochum', 'darmstadt_98', 'union_berlin', 'mainz_05',
        'koln', 'girona', 'real_madrid', 'barcelona', 'atl_madrid', 'athletic_club', 'betis',
        'real_sociedad', 'valencia', 'rayo_vallecano', 'las_palmas', 'getafe', 'osasuna',
        'villarreal', 'alaves', 'sevilla', 'cadiz', 'mallorca', 'celta_vigo', 'granada', 'almeria'
    ],
    'league_id': [0] * 20 + [1] * 20 + [2] * 20 + [3] * 16 + [4] * 22
}

teams_id = pd.DataFrame(teams_id)

In [137]:
teams_id

,team_id,team_name,league_id
0,0,botafogo_rj,0
1,1,palmeiras,0
2,2,bragantino,0
3,3,gremio,0
4,4,flamengo,0
...,...,...,...
93,93,cadiz,4
94,94,mallorca,4
95,95,celta_vigo,4
96,96,granada,4


In [138]:
historico = historico.merge(teams_id, left_on= 'team', right_on= 'team_name', how='left').drop(['team_name', 'league_id'], axis= 1)
historico = historico.rename(columns= {'team_id': 'home_team_id'})
historico = historico.merge(teams_id, left_on= 'opponent', right_on= 'team_name', how='left').drop(['team_name',], axis= 1)
historico = historico.rename(columns= {'team_id': 'away_team_id'})


In [139]:
historico.head()

,game_id,date,comp,round,venue,result,gf,ga,opponent,xg,...,fls,off,recov,season,team,days,time_diff,home_team_id,away_team_id,league_id
0,0,2023-04-15,Série A,1,Home,W,2,1,sao_paulo,0.9,...,20,2,57,2023,botafogo_rj,208,0.812207,0,8,0
1,1,2023-04-24,Série A,2,Away,W,2,1,bahia,0.2,...,9,5,54,2023,botafogo_rj,199,0.819550,0,14,0
2,2,2023-04-30,Série A,3,Away,W,3,2,flamengo,1.9,...,12,1,42,2023,botafogo_rj,193,0.824482,0,4,0
3,3,2023-05-07,Série A,4,Home,W,2,0,atl_mineiro,2.0,...,14,0,43,2023,botafogo_rj,186,0.830274,0,5,0
4,4,2023-05-11,Série A,5,Home,W,3,0,corinthians,2.6,...,12,0,54,2023,botafogo_rj,182,0.833601,0,12,0


### Divisão e Export

In [140]:
teams_id.to_excel(f'../datasets/xlsx/teams_id(tratado).xlsx')

In [141]:
historico.to_excel(f'../datasets/xlsx/historico(tratado).xlsx')

# TABELA

### Data Cleaning

In [142]:
tabela.drop(
  ['attendance', 'top team scorer', 'goalkeeper', 'notes', 'last 5'], 
  axis= 'columns', inplace= True)

In [143]:
tabela['squad'] = tabela['squad'].apply(remover_acentos)
tabela['squad'] = [linha.lower() for linha in tabela['squad']]
tabela['squad'] = tabela['squad'].str.replace(' ', '_')
tabela['squad'] = tabela['squad'].str.replace('(', '')
tabela['squad'] = tabela['squad'].str.replace(')', '')
tabela['squad'] = tabela['squad'].str.replace(')', '')

# Especifico para o brasileirão
tabela['squad'] = tabela['squad'].str.replace('atletico', 'atl')

# Especifico para a Premier League
tabela['squad'] = tabela['squad'].str.replace("united", "utd")
tabela['squad'] = tabela['squad'].str.replace("nott'ham_forest", "nottingham_forest")

# Especifico para a Bundesliga
tabela['squad'] = tabela['squad'].str.replace("m'gladbach", 'monchengladbach')

In [144]:
[item for item in tabela['squad'].unique() if item not in historico['opponent'].unique()]

[]

In [145]:
historico['opponent'].unique()

array(['sao_paulo', 'bahia', 'flamengo', 'atl_mineiro', 'corinthians',
       'goias', 'fluminense', 'america_mg', 'ath_paranaense', 'fortaleza',
       'cuiaba', 'palmeiras', 'vasco_da_gama', 'gremio', 'bragantino',
       'santos', 'coritiba', 'cruzeiro', 'internacional', 'botafogo_rj',
       'burnley', 'newcastle_utd', 'sheffield_utd', 'fulham', 'west_ham',
       'nottingham_forest', 'wolves', 'arsenal', 'brighton',
       'manchester_utd', 'bournemouth', 'brentford', 'liverpool',
       'luton_town', 'crystal_palace', 'chelsea', 'aston_villa',
       'tottenham', 'everton', 'manchester_city', 'monza', 'cagliari',
       'fiorentina', 'milan', 'empoli', 'sassuolo', 'salernitana',
       'bologna', 'torino', 'roma', 'atalanta', 'udinese', 'lazio',
       'lecce', 'hellas_verona', 'inter', 'genoa', 'juventus', 'napoli',
       'frosinone', 'rb_leipzig', 'monchengladbach', 'darmstadt_98',
       'bayern_munich', 'heidenheim', 'mainz_05', 'koln', 'wolfsburg',
       'freiburg', 'hoffe

### Feature Engineering

In [146]:
tabela['xg_conv'] = tabela['gf'] - tabela['xg']      #Conversão de xG
tabela['xga_conv'] = tabela['ga'] - tabela['xga']    #Conversão de xGA
tabela['att_rating'] = (tabela['gf']/tabela['mp'])/(tabela['gf']/tabela['mp']).mean()  #Media de gols por partida dividido pelo da liga
tabela['def_rating'] = (tabela['ga']/tabela['mp'])/(tabela['gf']/tabela['mp']).mean()  #Media de gols tomados por partida dividido pelo da liga
tabela['naive_rating'] = tabela['att_rating']*tabela['def_rating']*(tabela['gf']/tabela['mp']).mean()   #att*def*media da liga                                       #sh Total

In [147]:
tabela = tabela.merge(teams_id, left_on= 'squad', right_on= 'team_name', how='left').drop(['squad'], axis= 1)


### Divisões e Exports

In [148]:
tabela.to_excel(f'../datasets/xlsx/tabela(tratado).xlsx', index= False)

# RODADAS

### Data Cleaning

In [149]:
rodadas = rodadas.loc[rodadas['home'].notna()]

In [150]:
rodadas.drop(['day', 'date', 'time', 'attendance', 'venue', 'referee', 'match report', 'notes'], axis= 1, inplace= True)

In [151]:
rodadas['home'] = rodadas['home'].apply(remover_acentos)
rodadas['home'] = [linha.lower() for linha in rodadas['home']]
rodadas['home'] = rodadas['home'].str.replace(' ', '_')
rodadas['home'] = rodadas['home'].str.replace('(', '')
rodadas['home'] = rodadas['home'].str.replace(')', '')
rodadas['home'] = rodadas['home'].str.replace(')', '')

# Especifico para o brasileirão
rodadas['home'] = rodadas['home'].str.replace('atletico', 'atl')

# Especifico para a Premier League
rodadas['home'] = rodadas['home'].str.replace("nott'ham_forest", "nottingham_forest")

# Especifico para a Bundesliga
rodadas['home'] = rodadas['home'].str.replace("m'gladbach", 'monchengladbach')

In [152]:
rodadas['away'] = rodadas['away'].apply(remover_acentos)
rodadas['away'] = [linha.lower() for linha in rodadas['away']]
rodadas['away'] = rodadas['away'].str.replace(' ', '_')
rodadas['away'] = rodadas['away'].str.replace('(', '')
rodadas['away'] = rodadas['away'].str.replace(')', '')
rodadas['away'] = rodadas['away'].str.replace(')', '')

# Especifico para o brasileirão
rodadas['away'] = rodadas['away'].str.replace('atletico', 'atl')

# Especifico para a Premier League
rodadas['away'] = rodadas['away'].str.replace("nott'ham_forest", "nottingham_forest")

# Especifico para a Bundesliga
rodadas['away'] = rodadas['away'].str.replace("m'gladbach", 'monchengladbach')

In [153]:
[item for item in rodadas['home'].unique() if item not in historico['opponent'].unique()]

[]

In [154]:
rodadas[['gols_casa', 'gols_fora']] = rodadas['score'].str.split('–', expand=True)
rodadas.drop(['score'], axis= 1, inplace= True)
rodadas.insert(5, 'gols_casa', rodadas.pop('gols_casa'))
rodadas.insert(6, 'gols_fora', rodadas.pop('gols_fora'))

rodadas = rodadas.rename(columns={'xg.1': 'xg_fora', 'xg': 'xg_casa'})

rodadas['gols_casa'] = rodadas['gols_casa'].apply(convert_to_int)
rodadas['gols_fora'] = rodadas['gols_fora'].apply(convert_to_int)


In [155]:
rodadas = rodadas.merge(teams_id, left_on= 'home', right_on= 'team_name', how='left').rename(columns={'team_id': 'home_id'}).drop(['team_name', 'league_id'], axis= 1)
rodadas = rodadas.merge(teams_id, left_on= 'away', right_on= 'team_name', how='left').rename(columns={'team_id': 'away_id'}).drop(['team_name'], axis= 1)

In [156]:
rodadas.head()

,wk,home,xg_casa,xg_fora,away,gols_casa,gols_fora,league_name,home_id,away_id,league_id
0,1.0,america_mg,1.1,3.1,fluminense,0,3,Brasileirao,19,7,0
1,1.0,palmeiras,3.0,1.0,cuiaba,2,1,Brasileirao,1,11,0
2,1.0,bragantino,2.0,0.7,bahia,2,1,Brasileirao,2,14,0
3,1.0,botafogo_rj,0.9,2.0,sao_paulo,2,1,Brasileirao,0,8,0
4,1.0,ath_paranaense,0.8,1.0,goias,2,0,Brasileirao,6,17,0


In [157]:
rodadas = rodadas[['wk', 'xg_casa', 'xg_fora', 'home', 'gols_casa', 'gols_fora', 'away', 'league_name', 'home_id', 'away_id', 'league_id']]

### Divisão e Export

In [158]:
rodadas.to_excel(f'../datasets/xlsx/rodadas(tratado).xlsx', index= False)

# ELENCOS

In [159]:
elencos = elencos.drop(['nation', 'age', 'g+a', 'g-pk', 'pk', 'pkatt', 'npxg+xag', 'gls.1', 'ast.1', 'g+a.1',
                        'g-pk.1', 'g+a-pk', 'xg.1', 'xag.1', 'xg+xag', 'npxg.1', 'npxg+xag.1', 'matches'], axis= 1)

In [160]:
elencos['team'] = elencos['team'].apply(remover_acentos)
elencos['team'] = [linha.lower() for linha in elencos['team']]
elencos['team'] = elencos['team'].str.replace(' ', '_')
elencos['team'] = elencos['team'].str.replace('(', '')
elencos['team'] = elencos['team'].str.replace(')', '')
elencos['team'] = elencos['team'].str.replace(')', '')

# Especifico para o brasileirão
elencos['team'] = elencos['team'].str.replace('atletico', 'atl')
elencos['team'] = elencos['team'].str.replace('athletico', 'ath')
elencos['team'] = elencos['team'].str.replace('atletico', 'atl')

# Especifico para a Premier League
elencos['team'] = elencos['team'].str.replace('_hotspur', '')
elencos['team'] = elencos['team'].str.replace('rhampton_wanderers', 's')
elencos['team'] = elencos['team'].str.replace('united', 'utd')
elencos['team'] = elencos['team'].str.replace('west_ham_utd', 'west_ham')
elencos['team'] = elencos['team'].str.replace('_and_hove_albion', '')
elencos['team'] = elencos['team'].str.replace('wich_albion', '')
elencos['team'] = elencos['team'].str.replace('_town', '')
elencos['team'] = elencos['team'].str.replace('luton', 'luton_town')
elencos['team'] = elencos['team'].str.replace("nott'ham_forest", "nottingham_forest")

# Especifico para a Serie A TIM
elencos['team'] = elencos['team'].str.replace('nazionale', '')

elencos['team'] = elencos['team'].str.replace("m'gladbach", 'monchengladbach')
elencos['team'] = elencos['team'].str.replace("bayer_leverkusen", 'leverkusen')
elencos['team'] = elencos['team'].str.replace("eintracht_frankfurt", 'eint_frankfurt')

# Especifico para a La Liga
elencos['team'] = elencos['team'].str.replace("real_betis", 'betis')

In [161]:
[item for item in elencos['team'].unique() if item not in historico['opponent'].unique()]

[]

In [162]:
elencos = elencos.merge(teams_id, left_on= 'team', right_on= 'team_name', how='left').drop(['team_name'], axis= 1)

In [165]:
elencos.head()

,player,pos,mp,starts,min,90s,gls,ast,crdy,crdr,...,xag,prgc,prgp,prgr,season,team,league,coach,team_id,league_id
0,Lucas Perri,GK,32,32,2835.0,31.5,0.0,0.0,1.0,0.0,...,0.1,0.0,2.0,0.0,2023,botafogo_rj,Brasileirao,Lúcio Flávio,0,0
1,Adryelson,DF,30,30,2562.0,28.5,1.0,1.0,5.0,1.0,...,0.3,2.0,49.0,3.0,2023,botafogo_rj,Brasileirao,Lúcio Flávio,0,0
2,Víctor Cuesta,DF,29,29,2610.0,29.0,1.0,1.0,9.0,0.0,...,0.7,7.0,86.0,6.0,2023,botafogo_rj,Brasileirao,Lúcio Flávio,0,0
3,Marlon Freitas,MF,30,27,2440.0,27.1,1.0,1.0,3.0,0.0,...,1.6,24.0,176.0,16.0,2023,botafogo_rj,Brasileirao,Lúcio Flávio,0,0
4,Carlos Eduardo,MF,29,27,2352.0,26.1,5.0,5.0,5.0,0.0,...,5.5,28.0,139.0,80.0,2023,botafogo_rj,Brasileirao,Lúcio Flávio,0,0


In [176]:
elencos['pos'] = elencos['pos'].str.split(',').str[0]

In [177]:
elencos.to_excel(f'../datasets/xlsx/elencos(tratado).xlsx', index= False)